In [1]:
from finiteelementanalysis import visualize as viz
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
import numpy as np
import matplotlib.pyplot as plt

This notebook will show what happens when we take the small deformation case from the Euler beam solution comparison and apply a large load to it.

We will be using D2_nn6_tri with 100 horizontal elements and 2 vertical elements. Our beam is represented by a 30 x 1 rectangle.

In [2]:
ny = 3 # change this to refine the mesh
L = 30
H = 1
q = 500
E = 1e6
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])

for ele_type in ["D2_nn3_tri", "D2_nn6_tri"]:
    for nx in [5, 10, 20, 30, 50]:
        coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
        # Identify boundaries
        boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
        # 1. Fix left boundary: both u_x and u_y = 0.
        fixed_nodes_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
        fixed_nodes_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", 0.0, 0.0)
        fixed_nodes = np.hstack((fixed_nodes_left, fixed_nodes_right))
        # Assign distributed load on the right boundary
        dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, -q)
        # Assign artificial displacement field
        displacement = np.zeros((coords.shape))
        for kk in range(0, coords.shape[0]):
            displacement[kk, 0] = coords[kk, 0] * 0.01

        nr_num_steps = 5
        nr_print = True

        displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

        fname = f"large-def-{nx}-{ele_type}.gif"
        viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

Step 0, load factor = 0.200
Iteration 1, Correction=1.000000e+00, Residual=2.500000e+01, tolerance=1.000000e-09
Iteration 2, Correction=4.745078e-04, Residual=9.090497e+00, tolerance=1.000000e-09
Iteration 3, Correction=6.098020e-08, Residual=6.330951e-05, tolerance=1.000000e-09
Iteration 4, Correction=3.203739e-15, Residual=8.848634e-11, tolerance=1.000000e-09
Step 1, load factor = 0.400
Iteration 1, Correction=5.000420e-01, Residual=2.500000e+01, tolerance=1.000000e-09
Iteration 2, Correction=2.076800e-04, Residual=9.099869e+00, tolerance=1.000000e-09
Iteration 3, Correction=9.002170e-09, Residual=6.327290e-05, tolerance=1.000000e-09
Iteration 4, Correction=8.073340e-16, Residual=9.136226e-11, tolerance=1.000000e-09
Step 2, load factor = 0.600
Iteration 1, Correction=3.333504e-01, Residual=2.500000e+01, tolerance=1.000000e-09
Iteration 2, Correction=1.472196e-04, Residual=9.101520e+00, tolerance=1.000000e-09
Iteration 3, Correction=1.261719e-08, Residual=6.318114e-05, tolerance=1.000

![alt text](large-def-5-D2_nn3_tri.gif)
![alt text](large-def-10-D2_nn3_tri.gif)
![alt text](large-def-20-D2_nn3_tri.gif)
![alt text](large-def-5-D2_nn6_tri.gif)
![alt text](large-def-10-D2_nn6_tri.gif)
![alt text](large-def-20-D2_nn6_tri.gif)

Refining the mesh is very good if we start with a very coarse mesh with linear elements. Moving from linear to quadratic already moves the solver pretty much entirely to the "final" solution. In contrast adding more elements takes much longer.